## Cálculo da Entropia

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:

def Entropy(pA, pB):
  return - pA*np.log2(pA) - pB*np.log2(pB)

# Exemplo

In [ ]:
df = pd.read_csv('tabela.txt', sep='\s+')
df = df.set_index('Patient')
df

A entropia para esta divisão dos 14 pacientes em 5 para **Drug A** e 9 para **Drug B** corresponde a seguinte entropia.

In [ ]:
df['Drug'].value_counts()

In [ ]:
Entropy(5/14, 9/14)

## Seleção de uma variável de decisão para a árvore

A variável Cholesterol, tem como saída **Normal** e **High**.

Dos 5 **Drug A** temos 2 com Cholesterol **Normal** e 3 com Cholesterol **High**

Dos 9 **Drug B** temos 3 com Cholesterol **Normal** e 3 com Cholesterol **High**

In [ ]:
df.groupby(['Drug','Cholesterol']).size()

In [ ]:
# Cholesterol Normal
Entropy(2/8, 6/8)

In [ ]:
# Cholesterol High
Entropy(3/6, 3/6)

Variável **Sex** 

In [ ]:
df.groupby(['Drug','Sex']).size()

In [ ]:
# Sex = F
Entropy(4/7, 3/7)

In [ ]:
# Sex = M
Entropy(1/7, 6/7)

## Qual atributo fornece maior ganho de informação?

`O ganho de informação é o grau de certeza na classificação gerado por uma divisão nos dados.`

$$
\text{Ganho de Informação} = \text{Entropia antes da divisão} - \text{Entropia ponderada após a divisão}
$$

In [ ]:
# Ganho de informação para Sex

Entropy(5/14, 9/14) - ( 7/14 * Entropy(4/7, 3/7) + 7/14 * Entropy(1/7, 6/7) )

In [ ]:
# Ganho de informação para Cholesterol

Entropy(5/14, 9/14) - ( 8/14 * Entropy(2/8, 6/8) + 6/14 * Entropy(3/6, 3/6) )

O Maior ganho de informação vem através de Cholesterol! Esse teste deve ser feito para todas as colunas para encontrar a melhor partição.

## Usando o Scikit-Learn

In [ ]:
X = df.drop('Drug', axis=1)
y = df['Drug']

In [ ]:
X.columns

In [ ]:
dic = {}
for col in ('Age', 'Sex', 'BP', 'Cholesterol'):
    X[col], aux = pd.factorize( X[col] )
    dic[col] = {val: i for i,val in enumerate(aux)}    

In [ ]:
dic2 = {}
y, aux = pd.factorize( y )
dic2['Drug'] = {val: i for i,val in enumerate(aux)}  

In [ ]:
# Criar a árvore de decisão
model = DecisionTreeClassifier(criterion="entropy")

In [ ]:
# Treinar o modelo
clf = model.fit(X,y)

In [ ]:
# Gera a figura
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10), dpi=600)
tree.plot_tree(clf, filled=True, 
                feature_names = X.columns);

In [ ]:
imp = pd.DataFrame({'Feature': clf.feature_names_in_, 'Importância': clf.feature_importances_})
imp = imp.set_index('Feature')
imp.sort_values(by='Importância', ascending=False)

**Obs:** Esse dataset tem poucos pontos e não reflete a realidade

## Outra abordagem

In [ ]:
X = pd.get_dummies(df.drop('Drug', axis=1))

In [ ]:
# Treinar o modelo
clf = model.fit(X,y)

In [ ]:
# Gera a figura
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,6), dpi=600)
tree.plot_tree(clf, filled=True, 
                feature_names = X.columns);

In [ ]:
tree.export_graphviz(clf,
                     out_file="tree.dot",
                     feature_names = X.columns, 
                     filled = True)

https://dreampuf.github.io/GraphvizOnline/

# Random Forest

In [ ]:
# Load the Breast Cancer (Diagnostic) Dataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
# Arrange Data into Features Matrix and Target Vector
X = df.loc[:, df.columns != 'target']
y = df.loc[:, 'target'].values
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=0)
# Random Forests in `scikit-learn` (with N = 100)
rf = RandomForestClassifier(n_estimators=100,
                            random_state=0)
rf.fit(X_train, Y_train)

In [ ]:
rf.estimators_

In [ ]:
fn=data.feature_names
cn=data.target_names
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (10,10), dpi=800)
tree.plot_tree(rf.estimators_[0],
               feature_names = X.columns, 
               filled = True);

In [ ]:
# This may not the best way to view each estimator as it is small
fn=X.columns
#cn=data.target_names
fig, axes = plt.subplots(nrows = 1,ncols = 5,figsize = (10,2), dpi=3000)
for index in range(0, 5):
    tree.plot_tree(rf.estimators_[index],
                   feature_names = fn, 
                   #class_names=cn,
                   filled = True,
                   ax = axes[index]);
    
    axes[index].set_title('Estimator: ' + str(index), fontsize = 11)
fig.savefig('rf_5trees.png')